In [15]:
import pandas as pd
import numpy as np
import itertools
from collections import defaultdict
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import pairwise_distances

all_orders = pd.read_csv('all_except_last_orders.csv')
last_orders = pd.read_csv('last_orders_subset.csv')
print('All orders:', all_orders.shape, '| Last orders subset:', last_orders.shape)

All orders: (28984, 5) | Last orders subset: (5487, 5)


In [32]:
# Order–Item matrix
item_df= all_orders.copy()
item_df['order_count'] = 1 
item_df = item_df.groupby(['Order','SKU'])['order_count'].sum().reset_index()
item_pivot = item_df.pivot( index='Order',
                            columns='SKU',
                            values = "order_count" 
                          ).fillna( 0 ).reset_index(drop=True)

print('Matrix shape:', item_pivot.shape)
type(item_pivot)

Matrix shape: (2595, 632)


pandas.core.frame.DataFrame

In [41]:
all_orders.Member.value_counts()

Member
SWOZECO    347
SWLLREW    239
SWOEOHC    235
SSSZCHS    211
SWNORRH    206
          ... 
SWLLZEO      8
SWHONHS      8
SWEENCL      8
SWEEWEL      8
SSCRESN      8
Name: count, Length: 638, dtype: int64

In [17]:
# Association-Rule model
# Step 1: Set minimum support threshold
# This determines the minimum frequency of itemsets to be considered frequent
min_support = 0.01

# Step 2: Generate frequent itemsets using Apriori algorithm
# This finds all itemsets that appear together frequently in the transactions
# The 'use_colnames=True' parameter ensures we get actual item names instead of column indices
frequent_itemsets = apriori(item_pivot, min_support=min_support, use_colnames=True)

# Step 3: Generate association rules from frequent itemsets
# Using lift as the metric (lift > 1 indicates positive association)
# We set min_threshold=1.0 to only consider rules with positive association
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)

# Step 4: Prepare rules for recommendation
# Convert antecedents to frozenset for efficient lookup
# Extract single consequent item (assuming one-to-one recommendations)
rules['antecedents'] = rules['antecedents'].apply(lambda x: frozenset(x))
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

# Step 5: Create recommendation map
# Store recommendations in a dictionary for quick lookup
rec_map_ar = defaultdict(list)
for _, r in rules.sort_values('confidence', ascending=False).iterrows():
    rec_map_ar[r['antecedents']].append(r['consequents'])

# Step 6: Recommendation function
# Takes a cart and returns top k recommendations
def recommend_ar(cart, k=5):
    # Get the valid set of SKUs in the cart
    cart_set = frozenset(cart)

    # Get the list of popular SKUs for fallback recommendations
    pop = all_orders['SKU'].value_counts().index.tolist()

    # Initialize output list
    out = []

    # Generate recommendations from larger to smaller subsets of cart
    for sz in range(len(cart_set), 0, -1):
        # Try all combinations of current subset size
        for sub in itertools.combinations(cart_set, sz):
            sub = frozenset(sub)

            # If we have recommendations for this subset
            if sub in rec_map_ar:
                # Add recommendations that aren't already in cart or output
                for it in rec_map_ar[sub]:
                    if it not in cart_set and it not in out:
                        out.append(it)
                        if len(out) == k:
                            return out

    # If we haven't reached k recommendations yet, fall back to popular SKUs
    for it in pop:
        if it not in cart_set and it not in out:
            out.append(it)
        if len(out) == k: break

    return out

# Final Evaluation Score is coming at ~0.13

In [18]:
# Collaborative Filtering 
# Get the list of most popular SKUs
global_pop = all_orders['SKU'].value_counts().index.tolist()

def recommend_cf(cart, k=5):
    # Get the valid set of SKUs in the cart
    valid_cart = [s for s in set(cart) if s in item_sim.index]
    # valid_cart = [s for s in set(cart) if s in item_sim.index]
    
    # If the cart is empty, return the top k most popular SKUs
    if not valid_cart:
        return global_pop[:k]
    
    # Compute the average similarity score for each SKU in the cart
    scores = item_sim.loc[valid_cart].mean(axis=0)
    
    # Drop the SKUs in the cart from the scores
    scores = scores.drop(valid_cart, errors='ignore')
    
    # Sort the scores in descending order and return the top k SKUs
    return scores.sort_values(ascending=False).head(k).index.tolist()


In [19]:
# Validation util
def recall_at_k(pred, actual, k=5):
    """
    Calculate recall at k metric.

    Parameters:
    pred: List of recommended items
    actual: List of hidden items (ground truth)
    k: Number of top recommendations to consider

    Returns:
    Recall score (0.0 to 1.0) indicating how many hidden items were correctly recommended (TP/(TP+FN))
    """
    # Convert actual items to set for efficient lookup
    a = set(actual)
    # Count how many of the top k recommendations match the hidden items
    # Divide by the minimum of:
    # - Number of hidden items (since we can't recommend more than exist)
    # - k (since we're only looking at top k recommendations)
    return len([x for x in pred[:k] if x in a]) / min(len(a), k) if a else 0

def evaluate(fn, frac=0.3, seed=42):
    """
    Evaluate the recommendation system's performance using recall@k metric.

    Parameters:
    fn: The recommendation function to evaluate (e.g., recommend_ar or recommend_cf)
    frac: Fraction of orders to sample for evaluation
    seed: Random seed for reproducibility
    """
    # Set random seed to ensure reproducible results
    np.random.seed(seed)
    # Prepare the evaluation data
    # Group orders by their SKUs and convert to lists
    # Sample a fraction of orders for evaluation
    sampled = all_orders.groupby('Order')['SKU'].apply(list).sample(frac=frac, random_state=seed)
    # List to store recall scores for each order
    scores = []
    # Main evaluation loop over sampled orders
    for skus in sampled:
        # Skip orders with less than 2 items since we need at least one item to hide
        if len(skus) < 2: continue
        # Split the order into hidden and observed items
        # Hide approximately 1/3 of the items (minimum 1 item)
        # This simulates the scenario where we have partial cart information
        # and need to predict what other items the customer might buy
        hid = np.random.choice(skus, size=max(1, len(skus)//3), replace=False)
        # The remaining items become our "observed" cart
        # These are the items we'll use to generate recommendations
        obs = [x for x in skus if x not in hid]
        # Calculate recall for this order
        # fn(obs) generates recommendations based on the observed items
        # hid contains the items that were hidden (ground truth)
        # recall_at_k measures how many hidden items were correctly recommended
        scores.append(recall_at_k(fn(obs), hid))
    # Return the average recall across all sampled orders
    # If no valid scores were calculated (empty list), return 0.0
    return np.mean(scores) if scores else 0.0


In [20]:
## to be deleted before submission
import warnings
warnings.filterwarnings("ignore")

mthd = ['cosine', 'correlation', 'sokalmichener', 'canberra'
, 'yule', 'jaccard', 'seuclidean'
, 'matching', 'l1', 'manhattan', 'l2', 'cityblock', 'braycurtis'
, 'rogerstanimoto', 'dice', 'nan_euclidean', 'minkowski'
, 'russellrao', 'euclidean', 'sqeuclidean', 'chebyshev', 'hamming'
, 'sokalsneath']

best_distancte_metric = []                            
for i in mthd:
    item_sim = pd.DataFrame(1-pairwise_distances( item_pivot.T.to_numpy(), metric=i ) , 
                            index=item_pivot.columns, 
                            columns=item_pivot.columns) 
    s_cf = evaluate(recommend_cf)
    best_distancte_metric.append({"metric": i, "score": s_cf})
    print(i, ":", s_cf) 

best_distancte_metric = pd.DataFrame(best_distancte_metric)
best_distancte_metric = best_distancte_metric.sort_values(by='score', ascending=False).head(1)
best_metric = best_distancte_metric.metric.values[0]

print("best_metric:", best_metric, " Score:", best_distancte_metric.score.values[0])

cosine : 0.2781705227077978
correlation : 0.33541131105398464
sokalmichener : 0.00651242502142245
canberra : 0.00651242502142245
yule : 0.47688517566409594
jaccard : 0.2501285347043702
seuclidean : 0.006298200514138817
matching : 0.006833761782347901
l1 : 0.00651242502142245
manhattan : 0.00651242502142245
l2 : 0.006405312767780633
cityblock : 0.00651242502142245
braycurtis : 0.2571122536418166
rogerstanimoto : 0.00651242502142245
dice : 0.2571122536418166
nan_euclidean : 0.006405312767780633
minkowski : 0.006405312767780633
russellrao : 0.18221936589545842
euclidean : 0.006405312767780633
sqeuclidean : 0.00651242502142245
chebyshev : 0.002506426735218509
hamming : 0.006833761782347901
sokalsneath : 0.24907883461868038
best_metric: yule  Score: 0.47688517566409594


In [21]:
# # Item-CF model 

# best pair wise distance metric - Yule
# item_sim = pd.DataFrame(1-pairwise_distances( item_pivot.T.to_numpy(), metric=best_metric ) , 
#                         index=item_pivot.columns, 
#                         columns=item_pivot.columns)
# Final Evaluation Score is coming at 0.10559

# item_sim = 1 - pd.DataFrame(pairwise_distances( item_pivot.T.to_numpy(), metric="correlation" )) 
# Final Evaluation Score is coming at 0.08174

# item_sim = 1 - pd.DataFrame(pairwise_distances( item_pivot.T.to_numpy(), metric="jaccard" )) 
# Final Evaluation Score is coming at ~0.08

# Cosine Similarity
item_sim = pd.DataFrame(cosine_similarity(item_pivot.T), 
                        index=item_pivot.columns, 
                        columns=item_pivot.columns)
# Final Evaluation Score is coming at 0.17986

In [22]:
s_ar = evaluate(recommend_ar)
s_cf = evaluate(recommend_cf)
best = 'Collaborative Filtering' if s_cf > s_ar else 'Association Rule Method'
print(f'Recall @ 5 basket size – \n Association Rule Method: {s_ar:.4f} \n Collaborative Filtering: {s_cf:.4f}  \n Best: {best}')

Recall @ 5 basket size – 
 Association Rule Method: 0.1430 
 Collaborative Filtering: 0.2782  
 Best: Collaborative Filtering


In [23]:
# Generate final recommendations using the best method
# (either Association Rule Method or Collaborative Filtering)
rec_fn = recommend_ar if best=='Association Rule Method' else recommend_cf

# Initialize output list
rows = []

# Loop over each order in the last orders subset
# and generate recommendations for each order
for (member, order), g in last_orders.groupby(['Member','Order']):
    cart = g['SKU'].tolist()  # get the items in the order
    for sku in rec_fn(cart):  # generate recommendations for the order
        rows.append({'Member':member, 'Order':order, 'SKU':sku})  # append to output list

# Convert output list to DataFrame
out = pd.DataFrame(rows)

# Reset index and add 'ID' column
# (required for submission format)
out = out.reset_index(drop=False).rename(columns={'index':'ID'})
out['ID'] = (out['ID']+1).astype(int)

# Verify that the output file has the correct format
assert out['ID'].min()==1
assert out.groupby('Order').size().eq(5).all()

# Write output to file
out_file = f'Group_11_rec_5_sets.csv'
out.to_csv(out_file, index=False)
print('Wrote', out_file)

Wrote Group_11_rec_5_sets.csv


In [24]:
out.head()


,ID,Member,Order,SKU
0,1,SSCEHNS,8069966,15669780
1,2,SSCEHNS,8069966,15669878
2,3,SSCEHNS,8069966,15669772
3,4,SSCEHNS,8069966,15669760
4,5,SSCEHNS,8069966,15669777
